In [2]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import confusion_matrix,accuracy_score

import matplotlib.pyplot as plt
plt.style.use('ggplot')
%matplotlib inline

In [3]:
dataset_train=pd.read_csv('PM_train.txt',sep=' ',header=None).drop([26,27],axis=1)
col_names = ['id','cycle','setting1','setting2','setting3','s1','s2','s3','s4','s5','s6','s7','s8','s9','s10','s11','s12','s13','s14','s15','s16','s17','s18','s19','s20','s21']
dataset_train.columns=col_names
print('Shape of Train dataset: ',dataset_train.shape)
dataset_train.head()

FileNotFoundError: [Errno 2] No such file or directory: 'PM_train.txt'

In [ ]:
dataset_train['id'].value_counts()

In [ ]:
dataset_test=pd.read_csv('PM_test.txt',sep=' ',header=None).drop([26,27],axis=1)
dataset_test.columns=col_names
#dataset_test.head()
print('Shape of Test dataset: ',dataset_train.shape)
dataset_train.head()

In [ ]:
pm_truth=pd.read_csv('PM_truth.txt',sep=' ',header=None).drop([1],axis=1)
pm_truth.columns=['more']
pm_truth['id']=pm_truth.index+1
pm_truth.head()

In [ ]:
pm_truth.shape

In [ ]:
rul = pd.DataFrame(dataset_test.groupby('id')['cycle'].max()).reset_index()
rul.columns = ['id', 'max']
rul.head()

In [ ]:
rul.shape

In [ ]:
pm_truth['rtf']=pm_truth['more']+rul['max']
pm_truth.head()

In [ ]:
pm_truth.shape

In [ ]:
pm_truth.drop('more', axis=1, inplace=True)
dataset_test=dataset_test.merge(pm_truth,on=['id'],how='left')
dataset_test['ttf']=dataset_test['rtf'] - dataset_test['cycle']
dataset_test.drop('rtf', axis=1, inplace=True)
dataset_test.head()

In [ ]:
dataset_test.shape

In [ ]:
dataset_train['ttf'] = dataset_train.groupby(['id'])['cycle'].transform(max)-dataset_train['cycle']
dataset_train.head()

In [ ]:
dataset_train.shape

In [ ]:
dataset_train['ttf'].value_counts()

In [ ]:
df_train=dataset_train.copy()
df_test=dataset_test.copy()
period=30
df_train['label_bc'] = df_train['ttf'].apply(lambda x: 1 if x <= period else 0)
df_test['label_bc'] = df_test['ttf'].apply(lambda x: 1 if x <= period else 0)
df_train.head()

In [ ]:
df_train['label_bc'].value_counts()

In [ ]:
features_col_name=['setting1', 'setting2', 'setting3', 's1', 's2', 's3', 's4', 's5', 's6', 's7', 's8', 's9', 's10', 's11',
                   's12', 's13', 's14', 's15', 's16', 's17', 's18', 's19', 's20', 's21']
target_col_name='label_bc'

In [ ]:
sc=MinMaxScaler()
df_train[features_col_name]=sc.fit_transform(df_train[features_col_name])
df_test[features_col_name]=sc.transform(df_test[features_col_name])

In [ ]:
df_train.head()

In [ ]:
df_train.info()

In [ ]:
df_train['ttf'].min()

In [ ]:
df_train['ttf'].max()

In [ ]:
df_train.iloc[0,:]

In [ ]:
x_train = df_train.iloc[:,:-1].values
y_train = df_train.iloc[:,-1:].values

In [ ]:
from sklearn.linear_model import LogisticRegression
model = LogisticRegression()
model.fit(x_train,y_train)

In [ ]:
import joblib

In [ ]:


joblib.dump(model, "engine_model.sav")

In [ ]:
x_test = df_test.iloc[:,:-1].values
y_test = df_test.iloc[:,-1:].values

In [ ]:
y_predlog = model.predict(x_test)

In [ ]:
from sklearn.metrics import accuracy_score
accuracy_score(y_predlog,y_test)

In [ ]:
df_test['label_bc'].value_counts()

In [ ]:
from sklearn.metrics import confusion_matrix
cm1 = confusion_matrix(y_test,y_predlog)
cm1